<a href="https://colab.research.google.com/github/tr-dev-bc/Modern_CV_Assignments/blob/main/midterm24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt         # used to plot error later
# helps running in jupyter
%matplotlib inline

from torchvision import datasets, transforms

# Set device
device = "cuda" if torch.cuda.is_available() else "cpu"         # switch for gpu else cpu

# load and normalize the fmnist dataset
data_folder = '~/FMNIST'
fmnist = datasets.FashionMNIST(data_folder, download=True, train=True, transform=transform)
tr_images = fmnist.data
tr_targets = fmnist.targets

